# Scraping Pilpres 2024 Data

## Import Libraries & Setup

In [27]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
from datetime import datetime

In [28]:
# setup chrome options
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("--headless")  # ensure GUI is off
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")
chrome_options.page_load_strategy = 'normal'

## Functions

In [29]:
def get_data(province, url):
    # set up the webdriver
    driver = webdriver.Chrome(options=chrome_options)
    driver.get(url)
    
    table_xpath = "/html/body/div/div[1]/div/div[3]/div[2]/div[2]/div[2]/div/div/table"
    table_element = WebDriverWait(driver, 100).until(
        EC.presence_of_element_located((By.XPATH, table_xpath))
    )

    # Now that the table is loaded, extract the data
    # This example assumes the table rows are direct children of the table_element located by the XPath
    rows = table_element.find_elements(By.XPATH, ".//tr")
    data = []
    for row in rows:
        # Extract text from each cell in the row
        cells = row.find_elements(By.XPATH, ".//td")
        if not cells:  # If the first row consists of headers <th>
            cells = row.find_elements(By.XPATH, ".//th")
        data.append([cell.text for cell in cells])

    # Assuming the first row contains headers
    headers = data[0] if data else []
    data_rows = data[1:] if len(data) > 1 else []

    # Convert to DataFrame
    df = pd.DataFrame(data_rows, columns=headers)
    df['province'] = province
    df = df[df['Wilayah'] != 'Total\nProgress']
    # Clean Wilayah name
    df['Wilayah'] = df['Wilayah'].str.split('\n').str[0]
    
    return df

In [30]:
def process_data(df):
    # Rename columns
    new_column_names = ['kabupaten_kota','candidate_1_votes','candidate_2_votes','candidate_3_votes', 'province']
    df.columns = new_column_names
    
    # Add refreshed time
    current_time = datetime.now()
    df['refreshed_time'] = current_time
    
    # Remove DATA SEDANG DALAM PROSES to be null and correct the datatype
    columns_to_convert = ['candidate_1_votes', 'candidate_2_votes', 'candidate_3_votes']
    for col in columns_to_convert:
        df[col] = df[col].apply(lambda x: None if x == "Data sedang dalam proses" else x)
        df[col] = df[col].apply(lambda x: int(x.replace('.', '')) if x else None)

    # Add total votes column
    df['total_votes'] = df['candidate_1_votes'] + df['candidate_2_votes'] + df['candidate_3_votes']
    
    # Move refreshed_time & province to the left
    refreshed_column = df.pop('refreshed_time')
    province_column = df.pop('province')
    df.insert(0, 'province', province_column)
    df.insert(0, 'refreshed_time', refreshed_column)
    
    return df

## Dataset

In [31]:
# Link dataset for scraping kabupaten kota data
link_scraping = pd.read_csv('link_scraping.csv')
link_scraping['province'] = link_scraping['province'].str.split('\r').str[0]
# Replace spaces with empty string only for 'P A P U A'
link_scraping['province'] = link_scraping['province'].apply(lambda x: x.replace(' ', '') if x == 'P A P U A' else x)
link_scraping.head()

,province,link
0,ACEH,https://pemilu2024.kpu.go.id/pilpres/hitung-su...
1,BALI,https://pemilu2024.kpu.go.id/pilpres/hitung-su...
2,BANTEN,https://pemilu2024.kpu.go.id/pilpres/hitung-su...
3,BENGKULU,https://pemilu2024.kpu.go.id/pilpres/hitung-su...
4,DAERAH ISTIMEWA YOGYAKARTA,https://pemilu2024.kpu.go.id/pilpres/hitung-su...


## Main

In [32]:
# Table xpath national and per province are different
#national_table_xpath = "/html/body/div/div[1]/div/div[2]/div/div[2]/div[3]/div/div/table"
#province_table_xpath = "/html/body/div/div[1]/div/div[3]/div[2]/div[2]/div[2]/div/div/table"

To get the province data, sometimes we encounter some errors, so we wrote this code below

In [33]:
# Initialize an empty DataFrame
df = pd.DataFrame()

# to save if there is error rows
error_rows = []

for index, row in link_scraping.iterrows():
    try:
        temp_df = get_data(row['province'], row['link'])
        # Check if temp_df is not empty before concatenating
        if not temp_df.empty:
            df = pd.concat([df, temp_df], ignore_index=True)
            print(row['province'], "success")
        else:
            print(row['province'], "has no data")
    except Exception as e:
        print(f"Error processing {row['province']}: {str(e)}")
        error_rows.append(row)

# Retry processing the rows that encountered errors
for row in error_rows:
    try:
        temp_df = get_data(row['province'], row['link'])
        if not temp_df.empty:
            df = pd.concat([df, temp_df], ignore_index=True)
            print(row['province'], "success on retry")
        else:
            print(row['province'], "has no data on retry")
    except Exception as e:
        print(f"Error processing {row['province']} on retry: {str(e)}")

Error processing ACEH: Message: unknown error: net::ERR_CONNECTION_RESET
  (Session info: chrome-headless-shell=122.0.6261.95)
Stacktrace:
	GetHandleVerifier [0x00007FF6F93FAD22+56930]
	(No symbol) [0x00007FF6F936F622]
	(No symbol) [0x00007FF6F92242E5]
	(No symbol) [0x00007FF6F922011A]
	(No symbol) [0x00007FF6F92129F4]
	(No symbol) [0x00007FF6F9213D29]
	(No symbol) [0x00007FF6F9212CF3]
	(No symbol) [0x00007FF6F9211EF4]
	(No symbol) [0x00007FF6F9211E11]
	(No symbol) [0x00007FF6F9210625]
	(No symbol) [0x00007FF6F9210EDC]
	(No symbol) [0x00007FF6F92269FD]
	(No symbol) [0x00007FF6F92A8B47]
	(No symbol) [0x00007FF6F928BC9A]
	(No symbol) [0x00007FF6F92A81E2]
	(No symbol) [0x00007FF6F928BA43]
	(No symbol) [0x00007FF6F925D438]
	(No symbol) [0x00007FF6F925E4D1]
	GetHandleVerifier [0x00007FF6F9776AAD+3709933]
	GetHandleVerifier [0x00007FF6F97CFFED+4075821]
	GetHandleVerifier [0x00007FF6F97C817F+4043455]
	GetHandleVerifier [0x00007FF6F9499756+706710]
	(No symbol) [0x00007FF6F937B8FF]
	(No symbol)

In [34]:
df = df[df['province'].isin(link_scraping['province'])]

In [35]:
link_scraping.head()

,province,link
0,ACEH,https://pemilu2024.kpu.go.id/pilpres/hitung-su...
1,BALI,https://pemilu2024.kpu.go.id/pilpres/hitung-su...
2,BANTEN,https://pemilu2024.kpu.go.id/pilpres/hitung-su...
3,BENGKULU,https://pemilu2024.kpu.go.id/pilpres/hitung-su...
4,DAERAH ISTIMEWA YOGYAKARTA,https://pemilu2024.kpu.go.id/pilpres/hitung-su...


In [36]:
df.head()

,Wilayah,"H. ANIES RASYID BASWEDAN, Ph.D. - Dr. (H.C.) H. A. MUHAIMIN ISKANDAR",H. PRABOWO SUBIANTO - GIBRAN RAKABUMING RAKA,"H. GANJAR PRANOWO, S.H., M.I.P. - Prof. Dr. H. M. MAHFUD MD",province
0,BADUNG,5.805,126.652,101.614,BALI
1,BANGLI,1.951,56.887,56.082,BALI
2,BULELENG,14.786,159.975,101.240,BALI
3,GIANYAR,3.067,119.929,121.983,BALI
4,JEMBRANA,5.548,34.881,19.029,BALI


In [37]:
df = process_data(df)

In [38]:
df.head()

,refreshed_time,province,kabupaten_kota,candidate_1_votes,candidate_2_votes,candidate_3_votes,total_votes
0,2024-03-02 18:37:46.826249,BALI,BADUNG,5805.0,126652.0,101614.0,234071.0
1,2024-03-02 18:37:46.826249,BALI,BANGLI,1951.0,56887.0,56082.0,114920.0
2,2024-03-02 18:37:46.826249,BALI,BULELENG,14786.0,159975.0,101240.0,276001.0
3,2024-03-02 18:37:46.826249,BALI,GIANYAR,3067.0,119929.0,121983.0,244979.0
4,2024-03-02 18:37:46.826249,BALI,JEMBRANA,5548.0,34881.0,19029.0,59458.0


## Export Data

In [39]:
try:
    with open("data_pilpres_perprovince.csv", "w") as file:
        # If the file exists, overwrite it
        df.to_csv(file, index=False)
        print("File overwritten successfully.")

# If the file doesn't exist or there's another error, handle it
except FileNotFoundError:
    with open("data_pilpres_perprovince.csv", "w") as file:
        df.to_csv(file, index=False)
        print("New file created successfully.")
except Exception as e:
    print("An error occurred:", e)

File overwritten successfully.
